In [1]:
!pip install transformers sentence-transformers requests spotipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.9/268.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 32.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [2]:
import requests
import pandas as pd
import os
from transformers import pipeline
from sentence_transformers import SentenceTransformer
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from google.colab import userdata
import numpy as np
import requests


In [3]:
os.environ["SPOTIPY_CLIENT_ID"] = userdata.get("SPOTIPY_CLIENT_ID")
os.environ["SPOTIPY_CLIENT_SECRET"] = userdata.get("SPOTIPY_CLIENT_SECRET")
os.environ["SPOTIPY_REDIRECT_URI"] = "http://127.0.0.1:8888/callback"

In [4]:
class EmotionRecommender:
    def __init__(self):
        self.spotify = spotipy.Spotify(auth_manager=SpotifyOAuth(
            client_id=os.environ["SPOTIPY_CLIENT_ID"],
            client_secret=os.environ["SPOTIPY_CLIENT_SECRET"],
            redirect_uri=os.environ["SPOTIPY_REDIRECT_URI"],
            scope='user-read-private'
        ))

        self.available_genres = self._get_valid_genres()

        self.emotion_classifier = pipeline(
            "text-classification",
            model="joeddav/distilbert-base-uncased-go-emotions-student",
            top_k=None
        )
        self.semantic_model = SentenceTransformer('all-MiniLM-L6-v2')
        self.known_emotions = [
            'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring',
            'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval',
            'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude',
            'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride',
            'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral'
        ]

    def _get_valid_genres(self):
        """Get list of valid Spotify genres"""
        try:
            genres = self.spotify.recommendation_genre_seeds()['genres']
            return genres
        except Exception as e:
            print(f"Error getting genres: {str(e)}")
            return ['pop']

    def _get_songs(self, emotion):
        """Get song recommendations using random valid genre"""
        try:
            genre = random.choice(self.available_genres)
            recs = self.spotify.recommendations(
                seed_genres=[genre],
                limit=10
            )
            return [{
                "track": t["name"],
                "artist": t["artists"][0]["name"],
                "url": t["external_urls"]["spotify"]
            } for t in recs["tracks"]]
        except Exception as e:
            return [{"error": f"Spotify API Error: {str(e)}"}]

In [5]:
    def recommend(self, user_text):
        """Main recommendation method"""
        results = self.emotion_classifier(user_text)[0]
        emotions = {item['label']: item['score'] for item in results}
        primary = max(emotions, key=emotions.get)

        if primary not in self.known_emotions:
            mapped = self._expand_emotion(primary)
        else:
            mapped = primary

        return {
            "detected_emotion": mapped,
            "books": self._get_books(mapped),
            "songs": self._get_songs(mapped),
            "activity": self._get_activity(mapped)
        }

    def _expand_emotion(self, user_emotion):
        """Map unknown emotions using semantic similarity"""
        input_emb = self.semantic_model.encode([user_emotion])
        known_embs = self.semantic_model.encode(self.known_emotions)
        return self.known_emotions[np.argmax(np.dot(known_embs, input_emb.T))]

    def _get_books(self, emotion):
        """Get book recommendations from Open Library"""
        emotion_keywords = {
            "joy": "humor", "sadness": "inspiration",
            "anger": "self-help", "surprise": "mystery"
        }
        response = requests.get(
            f"https://openlibrary.org/search.json?q={emotion_keywords.get(emotion, 'fiction')}&limit=3"
        )
        return [{
            "title": doc.get("title", "Unknown Title"),
            "author": doc.get("author_name", ["Unknown Author"])[0],
            "year": doc.get("first_publish_year", "")
        } for doc in response.json().get("docs", [])[:3]]

    def _get_activity(self, emotion):
        """Get activity recommendation"""
        activities = {
            "joy": "Go for a walk in nature",
            "sadness": "Write in a journal",
            "anger": "Try meditation",
            "surprise": "Visit a new place"
        }
        return activities.get(emotion, "Read a book")


In [6]:
recommender = EmotionRecommender()

print("Please visit this URL to authenticate:", recommender.spotify.auth_manager.get_authorize_url())
redirect_response = input("Paste the redirect URL here: http://127.0.0.1:8888/callback")
code = recommender.spotify.auth_manager.parse_response_code(redirect_response)
recommender.spotify.auth_manager.get_access_token(code)

user_input = input("How are you feeling today? ")
results = recommender.recommend(user_input)

print("\nRecommendations:")
print(f"Detected Emotion: {results['detected_emotion']}")
print("\nBooks:")
for book in results['books']:
    print(f"- {book['title']} by {book['author']}")
print("\nSongs:")
for song in results['songs']:
    if "error" in song:
        print(f"- Error: {song['error']}")
    else:
        print(f"- {song['track']} by {song['artist']} ({song['url']})")
print(f"\nActivity: {results['activity']}")

KeyboardInterrupt: 